# Web Scraping Homework - Mission to Mars

In [1]:
# Import BeautifulSoup
from bs4 import BeautifulSoup as bs
# Import Pandas
import pandas as pd

In [2]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
# Create mars_data dictionary that we can insert into mongo
mars_data = {}

## NASA Mars News

In [4]:
# Visit the NASA Mars News Site 
url = "https://mars.nasa.gov/news/"
browser.visit(url)
browser.is_element_present_by_css(".list_text", 1)

True

In [5]:
# Scrape the browser into soup 
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# Find latest news title
div_text = soup.find('div', class_='list_text')
news_title = div_text.find('div', class_='content_title').a.text
print(news_title)

Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch


In [7]:
# Find latest news paragraph
news_p = div_text.find('div', class_='article_teaser_body').text
print(news_p)

A NASA Wallops Flight Facility cargo plane transported more than two tons of equipment — including the rover's sample collection tubes — to Florida for this summer's liftoff.


In [8]:
# Update mongo dictionary
mars_data['news_title'] = news_title
mars_data['news_paragraph'] = news_p

## JPL Mars Space Images - Featured Image

In [9]:
# Visit the following URL
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
browser.is_element_present_by_id("full_image", 1)

True

In [10]:
# Scrape the browser into soup 
html = browser.html
soup = bs(html, 'html.parser')

In [11]:
# Find first featured image and open link with full image
browser.find_by_id('full_image').first.click()
browser.is_element_present_by_css(".fancybox-image", 1)

True

In [12]:
# Scrape the browser into soup 
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
# Use soup to find full resolution image and save its url to variable `img_url`
img_url = soup.find("img", class_='fancybox-image')["src"]
featured_image_url = f"https://www.jpl.nasa.gov{img_url}"
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18884_ip.jpg


In [14]:
# Update mongo dictionary
mars_data['featured_image'] = featured_image_url

## Mars Weather

In [15]:
# Visit Mars Weather twitter account
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
## browser.is_element_present_by_css(".css-901oao.css-16my406.r-1qd0xha.r-ad9z0x.r-bcqeeo.r-qvutc0", 2)
browser.is_element_present_by_css(".css-901oao .css-16my406 .r-1qd0xha .r-ad9z0x .r-bcqeeo .r-qvutc0", 2)

False

In [16]:
# Scrape the browser into soup 
html = browser.html
soup = bs(html, 'html.parser')

In [17]:
# Use soup to find elements with weather tweets
w_div = soup.select('div.css-901oao.r-hkyrab.r-1qd0xha.r-a023e6.r-16dba41.r-ad9z0x.r-bcqeeo.r-bnwqim.r-qvutc0')
w_div

[<div class="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0" dir="auto" lang="en"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 529 (2020-05-22) low -93.2ºC (-135.8ºF) high 0.6ºC (33.1ºF)
 winds from the WNW at 4.6 m/s (10.3 mph) gusting to 15.5 m/s (34.6 mph)
 pressure at 7.10 hPa</span></div>,
 <div class="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0" dir="auto" lang="en"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 528 (2020-05-22) low -93.5ºC (-136.3ºF) high -5.3ºC (22.4ºF)
 winds from the SW at 6.3 m/s (14.0 mph) gusting to 23.6 m/s (52.8 mph)
 pressure at 7.10 hPa</span></div>,
 <div class="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0" dir="auto" lang="en"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 527 (2020-05-20) low -93.0ºC (-135.5ºF) high

In [18]:
# List the element with latest weather tweet
w_div[0]

<div class="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0" dir="auto" lang="en"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">InSight sol 529 (2020-05-22) low -93.2ºC (-135.8ºF) high 0.6ºC (33.1ºF)
winds from the WNW at 4.6 m/s (10.3 mph) gusting to 15.5 m/s (34.6 mph)
pressure at 7.10 hPa</span></div>

In [19]:
# Save the tweet text
mars_weather = w_div[0].span.text
print(mars_weather)

InSight sol 529 (2020-05-22) low -93.2ºC (-135.8ºF) high 0.6ºC (33.1ºF)
winds from the WNW at 4.6 m/s (10.3 mph) gusting to 15.5 m/s (34.6 mph)
pressure at 7.10 hPa


In [20]:
# Update mongo dictionary
mars_data['weather'] = mars_weather

## Mars Facts

In [21]:
# Visit Mars Facts webpage
url = 'https://space-facts.com/mars/'

In [22]:
# Use Pandas' `read_html` to parse the url
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [23]:
# Find Mars facts DataFrame in the list of DataFrames
# Assign the columns `['description', 'value']
mars_facts = tables[0]
mars_facts.columns = ['description', 'value']

mars_facts

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [24]:
# Set index to 'description' column
mars_facts.set_index('description', inplace=True)
mars_facts

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [25]:
# Save Dataframe to html table
html_mars_facts = mars_facts.to_html(justify='left')
html_mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: left;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

In [26]:
# Strip unwanted newlines to clean up the table
html_mars_facts = html_mars_facts.replace('\n', '')
html_mars_facts

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: left;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [27]:
# Update mongo dictionary
mars_data['facts'] = html_mars_facts

## Mars Hemispheres

In [28]:
# Visit USGS Astrogeology webpage
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

#### First loop to extract titles and links to pages with full images

In [29]:

# Design an XPATH selector to grab anchor with title
# and link to bring up the full resolution image
xpath = '//div[@class="description"]/a'
browser.is_element_present_by_xpath(xpath, 1)

# Find elements with titles and links to full images
results = browser.find_by_xpath(xpath)

# Initialize lists
titles = []
links = []

# loop for all 4 images
for i in range(len(results)):     
    # Set title
    titles.append(results[i].text)
    # Set links to pages with full image
    links.append(results[i]['href'])
                 
print(titles)
print(links)                

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']
['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


#### Second loop to extract in each iteration an image url string for the full resolution images. Save them with titles to a dictionary. Append the dictionaries to the list.

In [30]:
# Initialize list
hemisphere_image_urls = []

for i in range(len(links)):
    # Initialize loop dictionary
    img_dict = {}
    
    # Visist URLs
    browser.visit(links[i])
    browser.is_element_present_by_text("Sample", 1)
    
    # Use soup to find full resolution image
    html = browser.html
    soup = bs(html, 'html.parser')
    img_url = soup.find("a", text='Sample')['href']
    
    # Create dictionary for one hemisphere
    img_dict['title'] = titles[i]
    img_dict['img_url'] = img_url
    # Append it to list
    hemisphere_image_urls.append(img_dict) 
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [31]:
# Update mongo dictionary
mars_data['hemispheres'] = hemisphere_image_urls

In [32]:
browser.quit()

In [33]:
mars_data

{'news_title': "Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch",
 'news_paragraph': "A NASA Wallops Flight Facility cargo plane transported more than two tons of equipment — including the rover's sample collection tubes — to Florida for this summer's liftoff.",
 'featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18884_ip.jpg',
 'weather': 'InSight sol 529 (2020-05-22) low -93.2ºC (-135.8ºF) high 0.6ºC (33.1ºF)\nwinds from the WNW at 4.6 m/s (10.3 mph) gusting to 15.5 m/s (34.6 mph)\npressure at 7.10 hPa',
 'facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: left;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    